# Macro-data-prepare-FRED.ipynb

FRED の日次データなどを集める


In [6]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import io
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import datetime
from fred_database_utils import get_fred_ids_from_file, store_fred_database
from dateutil import relativedelta
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import sqlite3
from pathlib import Path
from etf_dot_com import get_etf_fundamentals, get_tickers_from_db, save_data_to_db
from tqdm import tqdm
import yfinance as yf
import curl_cffi

ROOT_DIR = Path().cwd().parent
FRED_DIR = ROOT_DIR / "data/FRED"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
series_id_list = get_fred_ids_from_file(file_path=FRED_DIR / "fred_series.json")
db_path = FRED_DIR / "FRED.db"
store_fred_database(db_path=db_path, series_id_list=series_id_list)


--- FREDデータの差分更新を開始 (データベース: 'FRED.db') ---
テーブル 'SP500' の最終日: 2025-09-26 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'SP500' を取得中 (開始日: 2025-09-27)...
	シリーズ 'SP500' に更新データはありませんでした。
テーブル 'CPIAUCSL' の最終日: 2025-08-01 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'CPIAUCSL' を取得中 (開始日: 2025-08-02)...
	シリーズ 'CPIAUCSL' に 1 件の新規データを追加しました。
テーブル 'DFF' の最終日: 2025-09-25 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'DFF' を取得中 (開始日: 2025-09-26)...
	シリーズ 'DFF' に更新データはありませんでした。
テーブル 'FEDFUNDS' の最終日: 2025-08-01 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'FEDFUNDS' を取得中 (開始日: 2025-08-02)...
	シリーズ 'FEDFUNDS' に 1 件の新規データを追加しました。
テーブル 'SOFR' の最終日: 2025-09-25 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'SOFR' を取得中 (開始日: 2025-09-26)...
	シリーズ 'SOFR' に更新データはありませんでした。
テーブル 'DPRIME' の最終日: 2025-09-25 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'DPRIME' を取得中 (開始日: 2025-09-26)...
	シリーズ 'DPRIME' に更新データはありませんでした。
テーブル 'DTB3' の最終日: 2025-09-25 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'DTB3' を取得中 (開始日: 2025-09-26)...
	シリーズ 'DTB3' に更新データはありませんでした。
テーブル 'DGS1MO' の最終日: 2025-09-25 

In [ ]:
conn = sqlite3.connect(db_path)
table_list = ["GDPC1", "CLVMNACSCAB1GQEA19", "JPNRGDPEXP"]
table_list = {
    "GDPC1": "Real GDP - US",
    "CLVMNACSCAB1GQEA19": "Real GDP - EUR",
    "JPNRGDPEXP": "Real GDP - Japan",
}
dfs = []
for table in table_list.keys():
    df = (
        pd.read_sql(f"SELECT * FROM `{table}`", con=conn, parse_dates="date")
        .drop_duplicates()
        .rename(columns={table: "value"})
        .assign(variable=table_list[table])
    )
    dfs.append(df)
gdp = pd.pivot(
    pd.concat(dfs, ignore_index=True), index="date", columns="variable", values="value"
)
display(gdp)
gdp_growth = gdp.pct_change(4).dropna()
gdp_growth.columns = [col + "(YoY)" for col in gdp_growth.columns]
display(gdp_growth)


variable,Real GDP - EUR,Real GDP - Japan,Real GDP - US
date,,,
1947-01-01,NaN,NaN,2182.681
1947-04-01,NaN,NaN,2176.892
1947-07-01,NaN,NaN,2172.432
1947-10-01,NaN,NaN,2206.452
1948-01-01,NaN,NaN,2239.682
...,...,...,...
2024-04-01,2825501.9,555861.7,23286.508
2024-07-01,2836980.0,557451.2,23478.570
2024-10-01,2848207.8,560774.2,23586.542


,Real GDP - EUR(YoY),Real GDP - Japan(YoY),Real GDP - US(YoY)
date,,,
1996-01-01,0.014164,0.032360,0.026013
1996-04-01,0.014192,0.035530,0.040024
1996-07-01,0.016459,0.024837,0.040501
1996-10-01,0.019925,0.033699,0.044215
1997-01-01,0.021368,0.027663,0.043141
...,...,...,...
2024-04-01,0.004726,-0.006295,0.031266
2024-07-01,0.009207,0.006997,0.027914
2024-10-01,0.012783,0.013492,0.023998


## Real GDP

データは[World Bank DataBank](https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.KD&country=#)より取得。GDP(constant 2015 US$)


In [ ]:
WORLD_BANK_DIR = ROOT_DIR / "data/World Bank"
gdp = pd.read_excel(WORLD_BANK_DIR / "world_bank-RealGDP.xlsx", sheet_name="Data")
display(gdp)


,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,GDP (constant 2015 US$),NY.GDP.MKTP.KD,Afghanistan,AFG,..,..,..,..,..,..,...,19134221644.732494,19566715174.163273,20084646751.495384,20323499020.363865,21118473702.347073,20621957125.207726,16345202562.548473,15325233810.138836,15672648238.100513,..
1,GDP (constant 2015 US$),NY.GDP.MKTP.KD,Albania,ALB,..,..,..,..,..,..,...,11470171826.957481,11918533446.974939,12309839868.708418,12761785703.107059,13025007474.889467,12593390460.142029,13722964212.461969,14385329993.534147,14951625324.018909,15543966722.070652
2,GDP (constant 2015 US$),NY.GDP.MKTP.KD,Algeria,DZA,27287513101.982952,23574926517.950321,18934192372.523987,25431219789.977432,26916253746.417416,28586918218.691067,...,187493855609.344635,194806115978.439453,197728207716.082001,200496402625.852692,202300870247.9198,192185826737.372894,199488888151.484772,206670488126.831696,215143978140.155029,222243729417.480896
3,GDP (constant 2015 US$),NY.GDP.MKTP.KD,American Samoa,ASM,..,..,..,..,..,..,...,673000000,661697709.923664,615461068.70229,631900763.358779,628818320.610687,656560305.343511,651422900.763359,662725190.839695,..,..
4,GDP (constant 2015 US$),NY.GDP.MKTP.KD,Andorra,AND,..,..,..,..,..,..,...,2789881258.503616,2893376871.723502,2903390035.737601,2949518094.678957,3008967036.982891,2672445970.886575,2893905656.789416,3170696511.929717,3252613194.161915,3362285466.981509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
